In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from player import Player
from dealer import Dealer
from game import TrucoGame
from environment import TrucoEnvironment
from actions import game_actions, game_actions_list
from agent import Agent
import random
import itertools
import torch as T
import numpy as np

In [2]:
device = T.device("cuda:0" if T.cuda.is_available() else "cpu")

In [4]:
num_players = 6
ids = list(range(0,num_players))


players = [Player(id) for id in ids]

agents = [Agent(player=players[i], state_space_dim=339, action_space_dim=game_actions.shape[0], device=device, replay_buffer_size=25000 + 5000*i, batch_size=64, target_update_freq=1500, epsilon_end=0.05, epsilon_decay=1000000) for i in range(6)]

random.shuffle(agents)

envs = [TrucoEnvironment([a1.player, a2.player]) for a1, a2 in zip(agents[0::2], agents[1::2])]

In [5]:
def get_env_by_agent(envs, agent):
    for env in envs:
        if agent.player in env.players:
            return env
    return None

def get_agents_by_env(agents, env):
    playing = []
    for agent in agents:
        if agent.player in env.players:
            playing.append(agent)
    return playing[0], playing[1]

In [6]:
for env in envs:
    
    playing_agents = get_agents_by_env(agents, env)
    
    for agent in playing_agents:

        # reset env
        starting_player, legal_actions, game_state = env.reset(False)

        # Initialize the ReplayBuffer
        while len(agent.replay_buffer) < agent.min_replay_size:
            # pick legal action given uniform distribution
            action = np.random.choice(legal_actions, 1, [1/len(legal_actions) for i in legal_actions])
            action = game_actions_list.index(action)

            # Take action, observer outcome
            rew, done, next_player, next_legal_actions, new_game_state = env.step(starting_player, action)

            #Save transition for training later
            transition = (game_state, action, rew, done, new_game_state)
            agent.save_transition(transition)

            starting_player = next_player
            legal_actions = next_legal_actions
            game_state = new_game_state

            if done:
                starting_player, legal_actions, game_state = env.reset()

In [7]:
PLAYER_SWAP_FREQ=50000

In [8]:
for epoch in itertools.count():
    print(f"Starting epoch: {epoch}")
    
    for env in envs:

        starting_player, legal_actions, game_state = env.reset(False, goes_first=1)
        playing_agents = get_agents_by_env(agents, env)

        # Main training loop
        for step in itertools.count():
            agent = [agent for agent in playing_agents if agent.player == starting_player][0]

            action = agent.choose_action(legal_actions, game_state)

            # Take action, observe outcome
            rew, done, next_player, next_legal_actions, new_game_state = env.step(agent.player, action)

            #Save transition for training later
            transition = (game_state, action, rew, done, new_game_state)
            agent.save_transition(transition)

            starting_player = next_player
            legal_actions = next_legal_actions
            game_state = new_game_state

            if done:
                # Give agents their episode reward
                for agent in playing_agents:
                    agent.save_reward(env.game.get_score(agent.player))

                starting_player, legal_actions, game_state = env.reset(False)

            # Learn from replay buffer
            agent.learn()

            if step % agent.save_freq == 0 and step > 0:
                agent.save_model()
                    

            # Logging
            if step % 25000 == 0 and step > 0:
                for agent in playing_agents:
                    avg_reward = np.mean(agent.reward_buffer)
                    print(f"Step: {step} | Player: {agent.player.get_id()} | Avg reward: {avg_reward} | W/L: {max([w if p == agent.player else 0 for p, w in env.games_won])/env.games_played} | Games: {env.games_played}")   

            # Break out of training loop when swap rate is reached
            if step % PLAYER_SWAP_FREQ == 0 and step > 0:
                print(f"Player {playing_agents[0].player.get_id()} vs Player {playing_agents[1].player.get_id()} finished.")
                
                # Clear GPU cache
                T.cuda.empty_cache()
                break;


                
    # Destroy all envs
    envs.clear()

    random.shuffle(agents)

    # Create new Envs
    envs = [TrucoEnvironment([a1.player, a2.player]) for a1, a2 in zip(agents[0::2], agents[1::2])]
    

Starting epoch: 0
Step: 25000 | Player: 2 | Avg reward: 0.9 | W/L: 0.5228159913851124 | Games: 7429
Step: 25000 | Player: 4 | Avg reward: 1.04 | W/L: 0.4771840086148876 | Games: 7429
Step: 50000 | Player: 2 | Avg reward: 0.82 | W/L: 0.5289778714436248 | Games: 14235
Step: 50000 | Player: 4 | Avg reward: 1.15 | W/L: 0.47102212855637515 | Games: 14235
Player 2 vs Player 4 finished.
Step: 25000 | Player: 0 | Avg reward: 1.0 | W/L: 0.5289267369559341 | Games: 7398
Step: 25000 | Player: 5 | Avg reward: 1.03 | W/L: 0.47107326304406594 | Games: 7398
Step: 50000 | Player: 0 | Avg reward: 0.9 | W/L: 0.5281552038100574 | Games: 14278
Step: 50000 | Player: 5 | Avg reward: 1.06 | W/L: 0.4718447961899426 | Games: 14278
Player 0 vs Player 5 finished.
Step: 25000 | Player: 1 | Avg reward: 1.09 | W/L: 0.5264279057382506 | Games: 7511
Step: 25000 | Player: 3 | Avg reward: 0.88 | W/L: 0.47357209426174945 | Games: 7511
Step: 50000 | Player: 1 | Avg reward: 1.1 | W/L: 0.527895504759258 | Games: 14393
Step

KeyboardInterrupt: 